# Intento de solución: taller uno de **Analítica Predictiva**

En este cuadernillo de Python se abordará el intento de solución del primer taller del curso de **Analítica Predictiva**.

En este documento se estará trabajando con una base de datos que contiene información sobre algunas características de los clientes de una entidad financiera, así como la historia de pagos y saldos de los últimos meses. El objetivo es predecir si el cliente entrará en mora al mes siguiente.

# 1. Limpieza de los datos

Realice la limpieza de los datasets:
- Renombre la columna "default payment next month" a "default".
- Remueva la columna "ID".
- Elimine los registros con informacion no disponible.
- Para la columna EDUCATION, valores > 4 indican niveles superiores
  de educación, agrupe estos valores en la categoría "others".
- Renombre la columna "default payment next month" a "default"
- Remueva la columna "ID".

### 1.0. Lectura de los datos

Se definirá una función que lea archivos CSV que estén comprimidos bajo un formato `ZIP`. Esto se realizará definiendo una función específica para ello. Luego, se imprimieran las primeras y las últimas tres observaciones de cada conjunto de datos.

In [1]:
import pandas as pd

pd.set_option('display.notebook_repr_html', False)


def importacion_comprimido(ruta: str) -> pd.DataFrame:
    """
    Esta función realiza la importación de un archivo con formato
    .zip y lo retorna como un marco de datos de pandas
    """

    df = pd.read_csv(
        ruta,
        header=0,
        sep=',',
        compression='zip',
    )

    return df

In [2]:
# Lectura del marco de datos de entrenamiento
df_tr = importacion_comprimido('../files/input/train_data.csv.zip')

# Lectura del marco de datos de validación
df_ts = importacion_comprimido('../files/input/test_data.csv.zip')

#
# Impresión de las primeras cinco filas
#
print(
    "Train:",
    pd.concat([df_tr.head(3), df_tr.tail(3)], axis=0),
    "",
    "###########################",
    "",
    "Test:",
    pd.concat([df_tr.head(3), df_ts.tail(3)], axis=0),
    sep='\n'
)

Train:
          ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  \
0      10748     310000    1          3         1   32      0      0      0   
1      12574      10000    2          3         1   49     -1     -1     -2   
2      29677      50000    1          2         1   28     -1     -1     -1   
20997     26      50000    1          3         2   23      0      0      0   
20998  14778      90000    2          3         2   25      0      0      0   
20999  20634     120000    1          2         2   31      0      0      0   

       PAY_4  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  \
0          0  ...      84373      57779      14163      8295      6000   
1         -1  ...       1690       1138        930         0         0   
2          0  ...      45975       1300      43987         0     46257   
20997      0  ...      28967      29829      30046      1973      1426   
20998      0  ...       5613      10113      10113      3000      300

### 1.1. Renombrar la columna default

In [3]:
def renombrar(df: pd.DataFrame, nombre_inicial: str, nombre_final: str) -> pd.DataFrame:
    """
    Esta función renombra el nombre de la columna de un marco de datos
    """
    df.rename(
        columns={nombre_inicial: nombre_final},
        inplace=True
    )

In [4]:
# Para entrenamiento:
renombrar(df_tr, 'default payment next month', 'default')
renombrar(df_ts, 'default payment next month', 'default')

# Observación
print(
    ">>> NOMBRE DE LAS COLUMNAS <<<",
    "TRAIN ----------",
    df_tr.columns,
    "",
    "#############",
    "",
    "TEST -----------",
    df_ts.columns,
    sep='\n'
)

>>> NOMBRE DE LAS COLUMNAS <<<
TRAIN ----------
Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')

#############

TEST -----------
Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')


### 1.2. Remover la columna `ID`

In [5]:
def remover_columna(df: pd.DataFrame, columna: str) -> pd.DataFrame:
    """
    Esta función elimina una de las columnas del marco de datos
    sin la necesidad de devolverla
    """
    df.pop(columna)

In [6]:
# Para entrenamiento:
remover_columna(df_tr, 'ID')

# Para validación
remover_columna(df_ts, 'ID')

# Observación
print(
    ">>> NOMBRE DE LAS COLUMNAS <<<",
    "TRAIN ----------",
    df_tr.columns,
    "",
    "#############",
    "",
    "TEST -----------",
    df_ts.columns,
    sep='\n'
)

>>> NOMBRE DE LAS COLUMNAS <<<
TRAIN ----------
Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')

#############

TEST -----------
Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')


### 1.3. Eliminación de registros sin información disponible

In [7]:
def revision_nulidad(df: pd.DataFrame) -> pd.DataFrame:
    """
    Esta función imprime la cantidad de registros nulos que se tienen
    por cada columna del marco de datos y luego los elimina
    """
    print(
        "Nulidad del marco de datos " + str(df),
        df.isna().sum()
    )

    df.dropna
    print(" √√√ Se han elimnado los registros nulos")


In [8]:
# Para entrenamiento
revision_nulidad(df_tr)

# Para validación
revision_nulidad(df_ts)

Nulidad del marco de datos        LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0         310000    1          3         1   32      0      0      0      0   
1          10000    2          3         1   49     -1     -1     -2     -1   
2          50000    1          2         1   28     -1     -1     -1      0   
3          80000    2          3         1   52      2      2      3      3   
4         270000    1          1         2   34      1      2      0      0   
...          ...  ...        ...       ...  ...    ...    ...    ...    ...   
20995     140000    2          2         1   27      2     -1     -1     -1   
20996     130000    1          2         2   41      0      0      0      0   
20997      50000    1          3         2   23      0      0      0      0   
20998      90000    2          3         2   25      0      0      0      0   
20999     120000    1          2         2   31      0      0      0      0   

       PAY_5  ...  BILL_

De acuerdo con lo anterior, ninguna de las variables tiene información nula; sin embargo, al revisar la documentación de la tabla, se observa que los valores `0` para `EDUCATION` y `MARRIAGE` se asocia con información nula (`N/A`), por lo que debe ser eliminada del marco de datos. Para ello, primero se convertiran estas ocurrencias en `NA` y luego se eliminarán.

In [9]:
def eliminacion_especial(
        df: pd.DataFrame, columna: str, valor: int
) -> pd.DataFrame:
    """
    Elimina los registros para los cuales una columna determinada
    toma ciertos valores numéricos entregados por el usuario
    """
    return df.loc[df[columna] != valor]
    

In [10]:
# Para entrenamiento
df_tr = eliminacion_especial(df=df_tr, columna='EDUCATION', valor=0)
df_tr = eliminacion_especial(df_tr, 'MARRIAGE', 0)

# Para validación
df_ts = eliminacion_especial(df_ts, 'EDUCATION', 0)
df_ts = eliminacion_especial(df_ts, 'MARRIAGE', 0)

print(
    "Registros en TRAIN:",
    str(len(df_tr)),
    "",
    "############",
    "",
    "Registros en TEST:",
    str(len(df_ts)),
    sep='\n',
)

Registros en TRAIN:
20953

############

Registros en TEST:
8979


### 1.4. Reemplazo de niveles

In [11]:
def reemplazo_nivel(
        df: pd.DataFrame, columna: str, nivel_inicial: int
) -> pd.DataFrame:
    """
    Esta función toma un marco de datos y una columna, modificando ciertos
    niveles
    """
    df[columna] = df[columna].apply(
        lambda x: nivel_inicial if x > nivel_inicial else x
    )

    return df

In [12]:
# Para entrenamiento
df_tr = reemplazo_nivel(df_tr, 'EDUCATION', 4)

# Para validación
df_ts = reemplazo_nivel(df_ts, 'EDUCATION', 4)

print(
    "Train:",
    df_tr['EDUCATION'].value_counts(),
    "",
    "#######",
    "",
    "Test:",
    df_ts['EDUCATION'].value_counts(),
    sep='\n'
)

Train:
EDUCATION
2    9756
1    7476
3    3396
4     325
Name: count, dtype: int64

#######

Test:
EDUCATION
2    4268
1    3105
3    1477
4     129
Name: count, dtype: int64


# 2. Conjuntos de entrenamiento y división

Para este caso no es necesario dividir los datos en conjuntos de entrenamiento y división, puesto que ya han sido entregados con esta división. Solo es necesario separar la variable respuesta de la matriz de diseño.

In [13]:
# Para el conjunto de entrenamieto
y_tr = df_tr['default']
X_tr = df_tr.drop(columns=['default'])

# Para el conjunto de validación
y_ts = df_ts['default']
X_ts = df_ts.drop(columns=['default'])

# 3. Definición de un *pipeline*

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import f_classif

In [15]:
# Transformación de variables categóricas
categoricas = ['SEX', 'EDUCATION', 'MARRIAGE']

# Preprocesador de columnas
preprocesador = ColumnTransformer(
    transformers=[
        ('categoricas', OneHotEncoder(), categoricas),
    ],
    remainder='passthrough'
)

# Pipeline
pipeline = Pipeline(
    [
        ('preprocesador', preprocesador),
        ('clf', RandomForestClassifier(random_state=44)),
    ],
)

# 4. Optimización de hiperparámetros

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score

In [17]:
# Grilla de hiperparámetros
param_grid = {
    'clf__n_estimators': [100],
    'clf__max_depth': [None],
    'clf__min_samples_split': [10],
    'clf__min_samples_leaf': [4],
    "clf__max_features": [23],
}


# Objeto optimizador
modelo = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=10,
    scoring='balanced_accuracy',
    n_jobs=10,
    refit=True,
    verbose=3,
)

Se procede con el ajuste:

In [18]:
modelo.fit(X_tr, y_tr)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocesador',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('categoricas',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('clf',
                                        RandomForestClassifier(random_state=44))]),
             n_jobs=10,
             param_grid={'clf__max_depth': [None], 'clf__max_features': [23],
                         'clf__min_samples_leaf': [4],
                         'clf__min_samples_split': [10],
                         'clf__n_estimators': [100]},
             scoring='balanced_accuracy', verbose=3)

In [19]:
modelo.best_params_

{'clf__max_depth': None,
 'clf__max_features': 23,
 'clf__min_samples_leaf': 4,
 'clf__min_samples_split': 10,
 'clf__n_estimators': 100}

# 5. Guardar el *pickle* del modelo

In [20]:
import pickle
import os

# Carpeta donde se guarda el modelo
direccion = '../files/models'
os.makedirs(direccion, exist_ok=True)

# Guardado del modelo
direc_modelo = '../files/models/model.pkl'
with open(direc_modelo, 'wb') as file:
    pickle.dump(modelo, file)

# 6. Cálculo de métricas

In [21]:
import json
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score

In [22]:
def calculo_metricas(
        modelo, y_tr, y_ts, X_tr, X_ts, directorio, nombre
):
    # Predicción
    y_pred_tr = modelo.predict(X_tr)
    y_pred_ts = modelo.predict(X_ts)

    # Métricas para entrenamiento
    metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'precision': precision_score(y_tr, y_pred_tr),
        'balanced_accuracy': balanced_accuracy_score(y_tr, y_pred_tr),
        'recall': recall_score(y_tr, y_pred_tr),
        'f1_score': f1_score(y_tr, y_pred_tr),
    }

    # Métricas para validcion
    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'precision': precision_score(y_ts, y_pred_ts),
        'balanced_accuracy': balanced_accuracy_score(y_ts, y_pred_ts),
        'recall': recall_score(y_ts, y_pred_ts),
        'f1_score': f1_score(y_ts, y_pred_ts),
    }

    # Almacenamiento
    os.makedirs(directorio, exist_ok=True)

    # Guardado de métricas
    with open(os.path.join(directorio, nombre), 'w') as file:
        file.write(json.dumps(metrics_train) + '\n')
        file.write(json.dumps(metrics_test) + '\n')

In [23]:
calculo_metricas(
    modelo=modelo,
    y_tr=y_tr,
    y_ts=y_ts,
    X_tr=X_tr,
    X_ts=X_ts,
    directorio='../files/output',
    nombre='metrics.json',
)

# 8. Matrices de confusión

In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
def format_confusion_matrix(matriz, conjunto):
    return {
        'type': 'cm_matrix',
        'dataset': conjunto,
        'true_0': {
            'predicted_0': int(matriz[0, 0]),
            'predicted_1': int(matriz[0, 1])
        },
        'true_1': {
            'predicted_0': int(matriz[1, 0]),
            'predicted_1': int(matriz[1, 1])
        },
    }

In [26]:
def matriz_confusion(
        modelo, y_tr, y_ts, X_tr, X_ts, directorio, nombre
):
    
    y_tr_pred = modelo.predict(X_tr)
    y_ts_pred = modelo.predict(X_ts)

    confusion_tr = confusion_matrix(y_tr, y_tr_pred)
    confusion_ts = confusion_matrix(y_ts, y_ts_pred)

    metrics = [
        format_confusion_matrix(confusion_tr, 'train'),
        format_confusion_matrix(confusion_ts, 'test'),
    ]

    # Guardar las matrices de confusión en el mismo archivo JSON
    with open(os.path.join(directorio, nombre), 'a') as f:  # Usar 'a' para agregar después de las métricas
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')

In [27]:
matriz_confusion(
    modelo=modelo,
    y_tr=y_tr,
    y_ts=y_ts,
    X_tr=X_tr,
    X_ts=X_ts,
    directorio='../files/output',
    nombre='metrics.json',
)

#

In [28]:
modelo.score(X_ts, y_ts)

np.float64(0.6772483895647385)